# Notebook for finding distribution of HAIL_MAX2D values

### Import necessary libraries- might need stats libraries later

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import norm
import matplotlib.mlab as mlab
import statistics

## Import WRF HIST, mid- and end-FUTR RCP 4.5, 8.5 files (resampled to daily max 12Z-12Z)

In [2]:
hist = xr.open_dataset('/home/scratch/jgoodin/hist_daily_max_hail_inches.nc') #Open hist daily max .nc file saved to /home/scratch directory
mid4p5 = xr.open_dataset('/home/scratch/jgoodin/mid_century_4p5_daily_max_hail_inches.nc') #Open mid-FUTR 4.5 daily max .nc file saved to /home/scratch directory
mid8p5 = xr.open_dataset('/home/scratch/jgoodin/mid_century_8p5_daily_max_hail_inches.nc') #Open mid-FUTR 8.5 daily max .nc file saved to /home/scratch directory
end4p5 = xr.open_dataset('/home/scratch/jgoodin/end_century_4p5_daily_max_hail_inches.nc') #Open end-FUTR 4.5 daily max .nc file saved to /home/scratch directory
end8p5 = xr.open_dataset('/home/scratch/jgoodin/end_century_8p5_daily_max_hail_inches.nc') #Open end-FUTR 8.5 daily max .nc file saved to /home/scratch directory

In [3]:
hist #Test print one dataset- make sure daily data are returned

<xarray.Dataset>
Dimensions:     (Time: 5479, south_north: 899, west_east: 1399)
Coordinates:
  * Time        (Time) datetime64[ns] 1990-09-30T12:00:00 ... 2005-09-29T12:0...
Dimensions without coordinates: south_north, west_east
Data variables:
    HAIL_MAX2D  (Time, south_north, west_east) float32 ...

## Resample by year and sum along the 'Time' dimension- provides summed values of HAIL_MAX2D for each grid for each simulation year. Take mean along 'year' dimension to provide annual mean HAIL_MAX2D value for each grid cell (15 years)

In [4]:
hist_annual = hist.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year') #Groupby year and sum HAIL_MAX2D values across time dim- take mean to find mean value 
# for each grid cell
mid4p5_annual = mid4p5.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year')
mid8p5_annual = mid8p5.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year')
end4p5_annual = end4p5.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year')
end8p5_annual = end8p5.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year')

In [8]:
hist_annual.HAIL_MAX2D.max() #Test print one epoch- should return array

<xarray.DataArray 'HAIL_MAX2D' ()>
array(3.57351828)

## Select values of HAIL_MAX2D

In [9]:
hist_annual = hist_annual.HAIL_MAX2D
mid4p5_annual = mid4p5_annual.HAIL_MAX2D
mid8p5_annual = mid8p5_annual.HAIL_MAX2D
end4p5_annual = end4p5_annual.HAIL_MAX2D
end8p5_annual = end8p5_annual.HAIL_MAX2D

## Convert HAIL_MAX2D data from each climate epoch to a 1D array (change variables to match climate epoch)

In [10]:

hist_annual_1d = hist_annual.values.ravel()
mid4p5_annual_1d = mid4p5_annual.values.ravel()
mid8p5_annual_1d = mid8p5_annual.values.ravel() #Use numpy's ravel method to return a contiguous flattened 1D array
end4p5_annual_1d = end4p5_annual.values.ravel()
end8p5_annual_1d = end8p5_annual.values.ravel()


## Remove all values from arrays ≠ 0

In [11]:
hist_nonzero = hist_annual_1d[hist_annual_1d != 0]
mid4p5_nonzero = mid4p5_annual_1d[mid4p5_annual_1d != 0]
mid8p5_nonzero = mid8p5_annual_1d[mid8p5_annual_1d != 0] #Select all values from climate epoch that are ≠ 0
end4p5_nonzero = end4p5_annual_1d[end4p5_annual_1d != 0] # **These are plotting variables**
end8p5_nonzero = end8p5_annual_1d[end8p5_annual_1d != 0]

In [18]:
hist_nonzero = hist_nonzero.reshape((-1, 1))
mid4p5_nonzero = mid4p5_nonzero.reshape((-1, 1))
mid8p5_nonzero = mid8p5_nonzero.reshape((-1, 1))
end4p5_nonzero = end4p5_nonzero.reshape((-1, 1))
end8p5_nonzero = end8p5_nonzero.reshape((-1, 1))

## Plot 1D data as line graph

In [20]:
fig = plt.figure(figsize = (8, 6))
hist_plt = plt.plot(hist_nonzero.T, color = 'mediumvioletred', label = "HIST")
mid4p5_plt = plt.plot(mid4p5_nonzero.T, color = 'teal', label = "mid-FUTR 4.5")
mid8p5_plt = plt.plot(mid8p5_nonzero.T, color = 'chocolate', label = "mid-FUTR 8.5")
end4p5_plt = plt.plot(end4p5_nonzero.T, color = 'seagreen', label = "end-FUTR 4.5")
end8p5_plt = plt.plot(end8p5_nonzero.T, color = 'goldenrod', label = "end-FUTR 8.5")
plt.grid(color = 'darkgray', linestyle = '--')
plt.legend()
plt.show()

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f4b6b1f39d0> (for post_execute):


KeyboardInterrupt: 